In [ ]:
import scanpy as sc
import pandas as pd
import anndata as ad
import numpy as np


In [ ]:
adata = sc.read_h5ad("/data/wuqinhua/phase/LungCancerCell/Lung_CancerCell_raw.h5ad")
adata

In [ ]:
adata.obs

In [ ]:
adata.var.index = adata.var["feature_name"]

In [ ]:
adata.obs["sample"].unique()

In [ ]:
adata.var.index

In [ ]:
# filter
# 1. filter tissue: lung
filtered_adata1 = adata[adata.obs["tissue"].isin(["lung"])].copy()

# 2. filter disease: normal & LUAD & LUSC
disease_types = ["lung adenocarcinoma", "normal", "squamous cell lung carcinoma"]
filtered_adata2 = filtered_adata1[filtered_adata1.obs["disease"].isin(disease_types)].copy()

# 3. filer origin: tumor_primary & normal
origin_types = ["tumor_primary", "normal"]
filtered_adata3 = filtered_adata2[filtered_adata2.obs["origin"].isin(origin_types)].copy()


filtered_adata3

In [ ]:
new_df = filtered_adata3.obs[['sample', 'age', 'ever_smoker','donor_id', 'uicc_stage', 'origin', 'disease', "tissue",'dataset']].drop_duplicates(subset='sample', keep='first')
new_df.to_csv('/data/wuqinhua/phase/LungCancerCell/sample_info.csv', index=False)

In [ ]:
new_df['disease'].value_counts()

In [ ]:
filtered_adata3.write_h5ad('/data/wuqinhua/phase/LungCancerCell/Lung_CancerCell.h5ad', compression="gzip")

In [ ]:
adata = sc.read_h5ad("/data/wuqinhua/phase/LungCancerCell/Lung_CancerCell.h5ad")
adata

In [ ]:
adata.obs = adata.obs.rename(columns={"sample": "sample_id", "disease": "phenotype", "ann_fine": "celltype"})

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=5000, flavor="cell_ranger", min_mean=0.0125, max_mean=3, min_disp=0.5, batch_key = 'dataset')
adata.raw = adata
adata_s = adata[:, adata.var.highly_variable].copy()

In [ ]:
sc.pl.umap(adata_s,color=["celltype"])

In [ ]:
sc.pl.umap(adata_s,color=["phenotype"])

In [ ]:
adata_s.var.index.name = None 
adata_s.write_h5ad("/data/wuqinhua/phase/LungCancerCell/Lung_CancerCell.h5ad", compression="gzip")